# Investigating the relationship between the 'basic_reproduction_number' parameter and Rt.

The effective reproduction number characterizes the temporal dynamics of an infectious disease.

Here I apply the bayesian inference method from branchpro to infer the effective R0 from simulation outputs.

In [12]:
import pandas as pd
import numpy as np
import branchpro
import scipy.stats
import statistics
import plotly.graph_objs as go

The main input parameter required by these models is the serial interval. We use serial intervals taken from https://github.com/SABS-R3-Epidemiology/transmission-heterogeneity-results/tree/main/data_library/serial_interval.

In [2]:
# Read serial interval
serial_interval = pd.read_csv("si-covid.csv", header=None)
serial_interval = serial_interval.fillna(0)
serial_intervals = serial_interval.values.T

We estimate the Rt profile for the simulations of New Zealand. The expected R0 in these simulations is 3.28. The output files are stored locally and are not provided on the GitHub repository due to their size.

In [3]:
df = pd.read_csv('../../../../NewZealand/output_NZ_relaxed.csv')

df = df.drop(["InfectionStatus.Exposed",
              "InfectionStatus.InfectASympt",
              "InfectionStatus.InfectGP",
              "InfectionStatus.InfectHosp",
              "InfectionStatus.InfectICU",
              "InfectionStatus.InfectICURecov",
              "InfectionStatus.Dead"],
              axis=1)
df = df.groupby(["time"]).agg({"InfectionStatus.Susceptible": 'sum',
                               "InfectionStatus.InfectMild": 'sum',
                               "InfectionStatus.Recovered": 'sum'})

In [10]:
x = df['InfectionStatus.Susceptible']
# first generate a list of infected cases
locally_infected_cases = [0] + list(-np.diff(x.values)[:30])
# then create a list of the number of imported cases per day
# in the New Zealand simulations one new case is introduced per day
imported = [1]*len(locally_infected_cases)
num_timepoints = len(locally_infected_cases)

# Set parameters of the prior on Rt and the sliding window
tau = 7
a = 1
b = 1/5

# Transform our incidence data into pandas dataframes
inc_data = pd.DataFrame(
    {
        'Time': np.arange(num_timepoints),
        'Incidence Number': locally_infected_cases
    }
)

imp_data = pd.DataFrame(
    {
        'Time': np.arange(num_timepoints),
        'Incidence Number': imported
    }
)

inference = branchpro.LocImpBranchProPosteriorMultSI(
    inc_data=inc_data, 
    imported_inc_data=imp_data,
    epsilon=1,
    daily_serial_intervals=serial_intervals,
    alpha=a,
    beta=b)

inference.run_inference(tau=tau)
intervals = inference.get_intervals(central_prob=.95)

In [11]:
fig = go.Figure()

trace1_relaxed = go.Scatter(
            y=intervals['Mean'],
            x=intervals['Time Points'],
            mode='lines',
            name='Estimated R',
            line_color='indigo'
        )

trace2_relaxed = go.Scatter(
    x=list(intervals['Time Points']) + list(intervals['Time Points'])[::-1],
    y=list(intervals['Lower bound CI']) + list(intervals['Upper bound CI'])[::-1],
    fill='toself',
    fillcolor='indigo',
    line_color='indigo',
    opacity=0.5,
    mode='lines',
    name='Credible interval 95%',
)

fig.add_trace(trace1_relaxed)
fig.add_trace(trace2_relaxed)

fig.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='Rt',
    hovermode='x unified')
fig.update_layout(plot_bgcolor='white', title='<b>a</b>', font=dict(size=12, color="Black"), yaxis_range=[0,22])
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.show()

first_week = intervals['Mean'][:7]

print('Average Rt over the first week = ', statistics.mean(first_week))


Average Rt over the first week =  5.1578682393961435


We then repeat the inference on the number of infections under more strict interventions.

In [6]:
df_strict = pd.read_csv('../../../../NewZealand/output_NZ_strict.csv')

df_strict = df_strict.drop(["InfectionStatus.Exposed",
              "InfectionStatus.InfectASympt",
              "InfectionStatus.InfectGP",
              "InfectionStatus.InfectHosp",
              "InfectionStatus.InfectICU",
              "InfectionStatus.InfectICURecov",
              "InfectionStatus.Dead"],
              axis=1)
df_strict = df_strict.groupby(["time"]).agg({"InfectionStatus.Susceptible": 'sum',
                               "InfectionStatus.InfectMild": 'sum',
                               "InfectionStatus.Recovered": 'sum'})

x_strict = df_strict['InfectionStatus.Susceptible']
# first generate a list of infected cases
locally_infected_cases_strict = [0] + list(-np.diff(x_strict.values)[:30])
# then create a list of the number of imported cases per day
# in the New Zealand simulations one new case is introduced per day
imported_strict = [1]*len(locally_infected_cases_strict)
num_timepoints_strict = len(locally_infected_cases_strict)

# Set parameters of the prior on Rt and the sliding window
tau = 3
a = 1
b = 1/5

# Transform our incidence data into pandas dataframes
inc_data_strict = pd.DataFrame(
    {
        'Time': np.arange(num_timepoints_strict),
        'Incidence Number': locally_infected_cases_strict
    }
)

imp_data_strict = pd.DataFrame(
    {
        'Time': np.arange(num_timepoints_strict),
        'Incidence Number': imported_strict
    }
)


inference_strict = branchpro.LocImpBranchProPosteriorMultSI(
    inc_data=inc_data_strict, 
    imported_inc_data=imp_data_strict,
    epsilon=1,
    daily_serial_intervals=serial_intervals,
    alpha=a,
    beta=b)


inference_strict.run_inference(tau=tau)
intervals_strict = inference_strict.get_intervals(central_prob=.95)
         

In [8]:
tau = 7
a = 1
b = 1/5

# Transform our incidence data into pandas dataframes
inc_data_strict = pd.DataFrame(
    {
        'Time': np.arange(num_timepoints_strict),
        'Incidence Number': locally_infected_cases_strict
    }
)

imp_data_strict = pd.DataFrame(
    {
        'Time': np.arange(num_timepoints_strict),
        'Incidence Number': imported_strict
    }
)


inference_strict = branchpro.LocImpBranchProPosteriorMultSI(
    inc_data=inc_data_strict, 
    imported_inc_data=imp_data_strict,
    epsilon=1,
    daily_serial_intervals=serial_intervals,
    alpha=a,
    beta=b)


inference_strict.run_inference(tau=tau)
intervals_strict = inference_strict.get_intervals(central_prob=.95)

In [9]:
fig_strict = go.Figure()

trace1 = go.Scatter(
            y=intervals_strict['Mean'],
            x=intervals_strict['Time Points'],
            mode='lines',
            name='Estimated R',
            line_color='indigo'
        )

trace2 = go.Scatter(
    x=list(intervals_strict['Time Points']) + list(intervals_strict['Time Points'])[::-1],
    y=list(intervals_strict['Lower bound CI']) + list(intervals_strict['Upper bound CI'])[::-1],
    fill='toself',
    fillcolor='indigo',
    line_color='indigo',
    opacity=0.5,
    mode='lines',
    name='Credible interval 95%',
)

fig_strict.add_trace(trace1)
fig_strict.add_trace(trace2)

fig_strict.update_layout(
    xaxis_title='Time (days)',
    yaxis_title='Rt',
    hovermode='x unified')
fig_strict.update_layout(plot_bgcolor='white', title='<b>b</b>', font=dict(size=12, color="Black"), yaxis_range=[0,22])
fig_strict.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig_strict.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig_strict.show()

first_week_strict = intervals_strict['Mean'][:7]

print('Average Rt over the first week = ', statistics.mean(first_week_strict))

Average Rt over the first week =  3.550159145046179


In [13]:
fig_strict.write_image('fig_strict.pdf')
fig.write_image('fig_relaxed.pdf')